In [1]:
import numpy as np
import pandas as pd
import os

from leo_vetter.stellar import quadratic_ldc
from leo_vetter.main import TCELightCurve

### Using Custom Vetting Thresholds

Pass-fail thresholds are meant to mimic human decision-making, but everyone has their own tolerance for what is considered a planet candidate. You may find that the current thresholds are too strong or weak.

You may also want to tune the pass-fail thresholds for your specific use-case. For example, if you are mostly interested in identifying new planets, you may want to use more lenient thresholds to recover more planets (at the cost of having a higher false positive/false alarm rate).

This example will briefly cover how you can tell the vetter to use a different set of thresholds.

In [2]:
tic = 231663901
per = 1.430363 
epo = 1338.885
dur = 0.069

# If you ran the previous notebooks, the metrics were saved to a file, so we can skip recomputing them here.
save_file = f"{tic}.1.metrics"

if os.path.exists(save_file):
    print("Loading saved metrics from file.")
    metrics = pd.read_csv(save_file, index_col=False)
    metrics = metrics.to_dict("records")[0]
else:
    # Download light curve
    search_result = lk.search_lightcurve(f"TIC {tic}", mission="TESS", author="TESS-SPOC")
    lcs = search_result.download_all()
    lc = lcs.stitch()
    lc = lc[~np.isnan(lc["flux"]) & (lc["quality"] == 0)]
    transit_mask = lc.create_transit_mask(transit_time=epo, period=per, duration=dur)
    lc_flat = lc.flatten(mask=transit_mask)

    time = np.asarray(lc_flat["time"].value)
    raw = np.asarray(lc["flux"].value)
    flux = np.asarray(lc_flat["flux"].value)
    flux_err = np.asarray(lc_flat["flux_err"].value)

    # Get stellar properties
    result = Catalogs.query_criteria(catalog="TIC", ID=tic)
    star = {}
    star["tic"] = tic
    for key in ["rad","mass","rho","Teff","logg"]:
        star[key] = float(result[key])
        star["e_"+key] = float(result["e_"+key])

    star["u1"], star["u2"] = quadratic_ldc(star["Teff"], star["logg"])

    # Load the TCELightCurve class
    tlc = TCELightCurve(tic, time, raw, flux, flux_err, per, epo, dur, planetno=1)
    
    # Flux-level vetting
    tlc.compute_flux_metrics(star, verbose=True)
    metrics = tlc.metrics

Loading saved metrics from file.


Open up the provided `user_thresholds.py` file and change any threshold you like. For example, you could change the function `large` to flag anything larger than 10 Earth radius as a false positive (the default is 25 Earth radius). You an also add new tests, as long as they meet the following criteria:

1. Take the metrics as an input
2. Outputs a mask (True if the test fails; False otherwise) and a message to be communicated if the test fails

Then, add the new test to the list of FA or FP tests in `check_thresholds`.

In [3]:
from user_thresholds import check_thresholds

FA = check_thresholds(metrics, "FA", verbose=True)
FP = check_thresholds(metrics, "FP", verbose=True)

Passed all FA tests
Passed all FP tests
